In [1]:
!pip install --upgrade pandas

Requirement already up-to-date: pandas in /home/nbuser/anaconda3_420/lib/python3.5/site-packages (0.24.2)
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pyarrow

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
from numpy import nan as NA
import os
import gc
from pandas.tseries.offsets import MonthEnd

from azure.storage.blob import BlockBlobService
import pyarrow.parquet as pq
from io import StringIO
from io import BytesIO

In [200]:
YearMonth = '2018-06-01'

In [201]:
#Azure account credentials
azure_storage_account_name = "agedatasciencecocstorage"
azure_storage_account_key = "V/FXG+lIWtYc854wbRA8BxGCLGJijS0NDwLvNHfmAxqRopAL3WHWGIQoHxpArG9lZoX+iY+dvtPYoB379w4MIw=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("You must provide a name and key for an Azure Storage account")

In [202]:
# First, we need to connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)
# There are also methods to list containers and blobs
#containers = blob_service.list_containers()
blobs = blob_service.list_blobs('banking-dm')

In [203]:
# List the blobs in the container.
print("\nList blobs in the container")
#generator = block_blob_service.list_blobs(container_name)
for blob in blobs:
    print("\t Blob name: " + blob.name)


List blobs in the container
	 Blob name: ailiev/Balances_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-05-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-06-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-01-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-02-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-03-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-04-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-05-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-05-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-0

### Read Demographic Data

In [204]:
container_name = "banking-dm"
blob_name_parquet = 'data/Customer_information_' + YearMonth + '.parquet.gzip'
blob_name = 'data/Customer_economic_sector_lookup.csv'

In [205]:
byte_stream = BytesIO()
try:
   blob_service.get_blob_to_stream(container_name=container_name, blob_name=blob_name_parquet, stream=byte_stream)
   cust_info = pq.read_table(source=byte_stream).to_pandas()
   # Do work on df ...
finally:
   # Add finally block to ensure closure of the stream
   byte_stream.close()

In [206]:
cust_info.head()

,Customer_ID,YearMonth,Birth_date,Customer_type,Customer_category,Gender,Marital_Status,Company_age,Customer_economic_sector,Customer_economic_branch
0,1,2018-06-01,1971-09-10,Individual,None,F,Divorced,None,ECO03,BR160
1,2,2018-06-01,1974-12-01,Professional,None,M,Married,None,ECO06,BR09
2,3,2018-06-01,1992-05-21,Individual,None,M,Married,None,ECO13,BR145
3,4,2018-06-01,None,Small Business,Retail,None,None,1977-11-08,ECO08,BR14
4,5,2018-06-01,1978-06-05,Individual,None,M,Single,None,ECO02,BR171


In [207]:
cust_info.shape

(100733, 10)

In [208]:
pd.value_counts(cust_info['Gender'], dropna=False)

M      44343
F      40900
NaN    15490
Name: Gender, dtype: int64

In [209]:
#read csv file
blobstring = blob_service.get_blob_to_text(container_name, blob_name).content
eco_sector = pd.read_csv(StringIO(blobstring), header=0, sep=';')

In [210]:
demographics = cust_info.merge(eco_sector, how='left', left_on='Customer_economic_sector', right_on='Customer_economic_sector_code')

In [211]:
del demographics['Customer_economic_sector']
del demographics['Customer_economic_sector_code']

In [212]:
demographics['YearMonth'] = pd.to_datetime(demographics['YearMonth'], format="%Y-%m-%d")  + MonthEnd(0)
demographics['Birth_date'] = pd.to_datetime(demographics['Birth_date'])
demographics['Company_age'] = pd.to_datetime(demographics['Company_age'])

In [213]:
demographics['Age_Months'] = np.where(pd.notnull(demographics['Birth_date']), 
                         (demographics['YearMonth'] - demographics['Birth_date'])/np.timedelta64(1,'M'),
                        (demographics['YearMonth'] - demographics['Company_age'])/np.timedelta64(1,'M'))

In [214]:
demographics['Age_Months'] = demographics['Age_Months'].astype('int')

In [215]:
del demographics['Birth_date']
del demographics['Company_age']
del demographics['YearMonth']
demographics['YearMonth'] = YearMonth

In [216]:
demographics = demographics[['YearMonth', 'Customer_ID', 'Customer_type', 'Customer_category', 'Gender',
       'Marital_Status', 'Customer_economic_branch', 'Sector', 'Age_Months']]

In [217]:
demographics.columns = ['YearMonth', 'Customer_ID', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY', 'GENDER',
       'MARITAL_STATUS', 'CUSTOMER_ECO_BRANCH', 'SECTOR', 'AGE_MONTHS']

In [218]:
demographics.head()

,YearMonth,Customer_ID,CUSTOMER_TYPE,CUSTOMER_CATEGORY,GENDER,MARITAL_STATUS,CUSTOMER_ECO_BRANCH,SECTOR,AGE_MONTHS
0,2018-06-01,1,Individual,None,F,Divorced,BR160,Arts and Communications,561
1,2018-06-01,2,Professional,None,M,Married,BR09,Finance,522
2,2018-06-01,3,Individual,None,M,Married,BR145,Manufacturing,313
3,2018-06-01,4,Small Business,Retail,None,None,BR14,Health and Science,487
4,2018-06-01,5,Individual,None,M,Single,BR171,Agriculture and Food,480


### Read Balances Data

In [219]:
blob_name_parquet_bm = 'ailiev/Balances_Master_' + YearMonth + '.parquet.gzip'

In [220]:
byte_stream = BytesIO()
try:
   blob_service.get_blob_to_stream(container_name=container_name, blob_name=blob_name_parquet_bm, stream=byte_stream)
   bal_master = pq.read_table(source=byte_stream).to_pandas()
   # Do work on df ...
finally:
   # Add finally block to ensure closure of the stream
   byte_stream.close()

In [221]:
mcif = demographics.merge(bal_master, how='left', on=[ 'YearMonth', 'Customer_ID'])

In [222]:
mcif.head()

,YearMonth,Customer_ID,CUSTOMER_TYPE,CUSTOMER_CATEGORY,GENDER,MARITAL_STATUS,CUSTOMER_ECO_BRANCH,SECTOR,AGE_MONTHS,CNT_AL_PROMOTION,...,PCT_VISA_PLATINUM_CNT,PCT_NEW_HOUSE_CNT,PCT_RENOVATION_CNT,PCT_OVD_CNT,PCT_PL_F_NON_SEC_CNT,PCT_PL_F_SEC_CNT,PCT_PL_V_CNT,PCT_SA_CNT,PCT_TD_LT_CNT,PCT_TD_ST_CNT
0,2018-06-01,1,Individual,None,F,Divorced,BR160,Arts and Communications,561,NaN,...,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-06-01,2,Professional,None,M,Married,BR09,Finance,522,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-06-01,3,Individual,None,M,Married,BR145,Manufacturing,313,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-06-01,4,Small Business,Retail,None,None,BR14,Health and Science,487,NaN,...,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-06-01,5,Individual,None,M,Single,BR171,Agriculture and Food,480,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [223]:
del demographics, bal_master
gc.collect()

59

### Read Transactions Data

In [224]:
blob_name_parquet_tr = 'ailiev/Transactions_Master_' + YearMonth + '.parquet.gzip'

In [225]:
byte_stream = BytesIO()
try:
   blob_service.get_blob_to_stream(container_name=container_name, blob_name=blob_name_parquet_tr, stream=byte_stream)
   transactions_master = pq.read_table(source=byte_stream).to_pandas()
   # Do work on df ...
finally:
   # Add finally block to ensure closure of the stream
   byte_stream.close()

In [226]:
mcif = mcif.merge(transactions_master, how='left', on=[ 'YearMonth', 'Customer_ID'])

In [227]:
mcif.head()

,YearMonth,Customer_ID,CUSTOMER_TYPE,CUSTOMER_CATEGORY,GENDER,MARITAL_STATUS,CUSTOMER_ECO_BRANCH,SECTOR,AGE_MONTHS,CNT_AL_PROMOTION,...,PCT_TNAM_DT,PCT_NUTN_CC,PCT_NUTN_DC,PCT_NUTN_LD,PCT_TNAM_CC,PCT_TNAM_DC,PCT_TNAM_LD,PCT_DAYS_TR_CC,PCT_DAYS_TR_DC,PCT_DAYS_TR_LD
0,2018-06-01,1,Individual,None,F,Divorced,BR160,Arts and Communications,561,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-06-01,2,Professional,None,M,Married,BR09,Finance,522,NaN,...,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,2018-06-01,3,Individual,None,M,Married,BR145,Manufacturing,313,NaN,...,0.950897,NaN,0.393939,0.606061,NaN,0.210289,0.789711,NaN,0.5,0.700000
3,2018-06-01,4,Small Business,Retail,None,None,BR14,Health and Science,487,NaN,...,0.711657,0.954545,NaN,0.045455,0.711657,NaN,0.288343,0.956522,NaN,0.086957
4,2018-06-01,5,Individual,None,M,Single,BR171,Agriculture and Food,480,NaN,...,0.788336,0.928571,NaN,0.071429,0.788336,NaN,0.211664,0.944444,NaN,0.111111


In [228]:
def write_pandas_dataframe_to_blob(blob_service, df, container_name, blob_name):
    """ Write Pandas dataframe to blob storage """
    buffer = BytesIO()
    df.to_parquet(buffer)
    blob_service.create_blob_from_bytes(
        container_name=container_name, blob_name=blob_name, blob=buffer.getvalue()
    )

In [229]:
write_to_blob_parquet = 'ailiev/MCIF_' + YearMonth + '.parquet.gzip'

In [230]:
# Write to blob using pyarrow
write_pandas_dataframe_to_blob(blob_service, mcif, 'banking-dm', write_to_blob_parquet)

In [231]:
del transactions_master, mcif
gc.collect()

16